## Environment Setup

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# downloading packages for running the notebook
import sys
import subprocess



In [3]:
!pip install sentence-transformers

In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from sentence_transformers import SentenceTransformer

m2 = SentenceTransformer('sentence-transformers/LaBSE')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### OpenAI Setup

Add openai key for gpt-3.5 model usage.

In [18]:
import openai

api_key = 'aa-m9TQcsmZMErvs4oFI0Si7zcCZCBj1oxTn92Ikperiy7oOfS1'

### Dataset Cleaning and Preprocessing Steps

In [19]:
with open('fa.txt', 'r') as f:
    data = f.read()

data = data.replace('\n\n','\n')

In [20]:
ques_ans = dict()


In [21]:
import pandas as pd

qa_dict = dict()

qa_dict['text'] = []

import numpy as np

# تنظیم تعداد خطوطی که باید با هم ترکیب شوند
lines_per_group = 3
grouped_sentences = []

# ترکیب خطوط و ایجاد گروه‌ها
temp_text = ""
for i, line in enumerate(data.split("\n")):
    temp_text += line + " "
    # هر ده خط یک بار به لیست اضافه می‌شود
    if (i + 1) % lines_per_group == 0:
        grouped_sentences.append(temp_text.strip())
        temp_text = ""

# در صورتی که خطوط باقی‌مانده‌ای وجود داشته باشد، آن‌ها را نیز اضافه کنید
if temp_text:
    grouped_sentences.append(temp_text.strip())
qa_dict['text']=grouped_sentences





In [22]:
from datasets import Dataset
qa_df = pd.DataFrame.from_dict(qa_dict)
# You can load a Dataset object this way
dataset = Dataset.from_pandas(qa_df)

dataset

Dataset({
    features: ['text'],
    num_rows: 22872
})

### Text Preprocessing Functions

In [23]:
## Embedding Generation Function

import numpy as np
def sent_check2(example):

    output = np.zeros([768,], dtype=np.float32)
    try:
        sent_embed = m2.encode(example['text'])
        output = sent_embed
    except:
        print("Error")

    example['embeddings'] = output

    return example

In [24]:

dataset = dataset.map(sent_check2)

Map:   0%|          | 0/22872 [00:00<?, ? examples/s]

In [13]:
from datasets import Dataset

# Assuming your dataset is already created with 'text' and 'embeddings' columns
dataset.save_to_disk('./')


Saving the dataset (0/1 shards):   0%|          | 0/22872 [00:00<?, ? examples/s]

In [26]:
pip install faiss-gpu

In [1]:
from datasets import load_from_disk

# Load the dataset
dataset = load_from_disk('./')


In [2]:
dataset.add_faiss_index(column='embeddings')

  0%|          | 0/23 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 22872
})

In [35]:
question = "افردادی که اموال خود را در راه خدا انفاق میکنند شبیه چه هستند؟"

question_embedding = m2.encode(question)
scores, retrieved_examples = dataset.get_nearest_examples('embeddings', question_embedding, k=10)
print(scores)
for exp in retrieved_examples['text']:  # Accessing the first result set

    print(exp)

    print("*****************************")

[0.8905533  0.90861076 0.92659897 0.93502325 0.9378055  0.9396076
 0.94623244 0.95348036 0.9626346  0.9631235 ]
آنان كه بخل مى‌ورزند و مردم را به بخل وا مى‌دارند و مالى را كه خدا به آنها داده است پنهان مى‌كنند. و ما براى كافران عذابى خواركننده مهيا ساخته‌ايم. و نيز كسانى را كه اموال خويش براى خودنمايى انفاق مى‌كنند و به خدا و روز قيامت ايمان نمى‌آورند. و هر كه شيطان قرين او باشد، قرينى بد دارد. چه زيان داردشان اگر به خدا و روز قيامت ايمان آورند و از آنچه خدا به آنها روزى داده است انفاق كنند؟ خدا به كارشان داناست.
*****************************
آیا در آنچه خداوند آفریده است نمی‌نگرند که سایه‌هایشان از راست و چپ می‌گسترد و فروتنانه به درگاه خداوند سجده می‌کنند؟ هر آن جنبنده‌ای که در آسمانها و آنچه در زمین است، و فرشتگان به درگاه خدا سجده می‌برند و گردنکشی نمی‌کنند از پروردگارشان، که بر آنان محیط است، می‌ترسند و آنچه دستور یافته‌اند، انجام می‌دهند
*****************************
آيا آب دادن به حاجيان و آباد كردن مسجد الحرام را مانند [كار] كسى قرار داده‌ايد كه به خدا و روز واپسين ايمان آورده 

In [11]:
question = "هنگامی که اصحاب موسی به یک نوع طعام اعتراض کردند موسی به آنها چه گفت؟"

question_embedding = m2.encode(question)
scores, retrieved_examples = dataset.get_nearest_examples('embeddings', question_embedding, k=10)

In [32]:
# Assuming hits is a list of dictionaries with 'corpus_id' as a key
for exp in retrieved_examples['text']:  # Accessing the first result set

    print(exp)

    print("*****************************")



(به خاطر بیاور) هنگامی را که پروردگارت موسی را ندا داد که به سراغ قوم ستمگر برو... قوم فرعون، آیا آنان (از مخالفت فرمان پروردگار) پرهیز نمی‌کنند؟! (موسی) عرض کرد: «پروردگارا! از آن بیم دارم که مرا تکذیب کنند،
*****************************
گفتند: ما همچنان بر پرستش آن خواهيم بود تا موسى به نزد ما بازگردد موسى گفت: اى هارون! وقتى ديدى آنها گمراه شدند چه چيز مانع تو شد كه از من پيروى نكنى؟ آيا امر مرا نافرمانى كردى
*****************************
[هنگامی که موسی بازگشت، به هارون] گفت: ای هارون! وقتی دیدی آنان گمراه شدند چه چیز تو را مانع شد، از اینکه مرا [در برخورد شدید با گمراهان] پیروی کنی؟ آیا از فرمان من سرپیچی کردی؟ گفت: ای پسر مادرم! نه ریش مرا بگیر و نه سرم را، من ترسیدم که بگویی: میان بنی اسرائیل تفرقه و جدایی انداختی و سفارش مرا [در حفظ وحدت بنی اسرائیل] رعایت نکردی.
*****************************
[موسى‌] گفت: «اى هارون، وقتى ديدى آنها گمراه شدند چه چيز مانع تو شد، كه از من پيروى كنى؟ آيا از فرمانم سر باز زدى؟» گفت: «اى پسر مادرم، نه ريش مرا بگير و نه [موى‌] سرم را، من ترسيدم بگويى:

In [7]:
scores

array([0.8863036 , 0.9180075 , 0.93875974, 0.9597673 , 0.9651403 ,
       0.9710073 , 0.9719907 , 0.9732956 , 0.98156464, 0.9855365 ],
      dtype=float32)

In [25]:

import torch
from sentence_transformers import util

# Ensure that question_embedding is a tensor
question_embedding_tensor = torch.tensor([question_embedding])

# Convert dataset['embeddings'] to a tensor if it's a list
embeddings_tensor = torch.tensor(dataset['embeddings'])
hits = util.semantic_search(question_embedding, embeddings_tensor)
hits

<ipython-input-25-52f49fedcaed>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  question_embedding_tensor = torch.tensor([question_embedding])


[[{'corpus_id': 15531, 'score': 0.5766130685806274},
  {'corpus_id': 17442, 'score': 0.5402070879936218},
  {'corpus_id': 7049, 'score': 0.5390711426734924},
  {'corpus_id': 13285, 'score': 0.5205036997795105},
  {'corpus_id': 4970, 'score': 0.519374668598175},
  {'corpus_id': 15030, 'score': 0.5182271003723145},
  {'corpus_id': 9295, 'score': 0.516907811164856},
  {'corpus_id': 15033, 'score': 0.5148196816444397},
  {'corpus_id': 15289, 'score': 0.5141289830207825},
  {'corpus_id': 11272, 'score': 0.5137339234352112}]]

In [30]:
# Assuming hits is a list of dictionaries with 'corpus_id' as a key
for hit in hits[0]:  # Accessing the first result set
    corpus_id = hit['corpus_id']
    print(dataset["text"][corpus_id])
    print(hit['score'])
    print("*****************************")



(به خاطر بیاور) هنگامی را که پروردگارت موسی را ندا داد که به سراغ قوم ستمگر برو... قوم فرعون، آیا آنان (از مخالفت فرمان پروردگار) پرهیز نمی‌کنند؟! (موسی) عرض کرد: «پروردگارا! از آن بیم دارم که مرا تکذیب کنند،
0.5766130685806274
*****************************
گفتند: ما همچنان بر پرستش آن خواهيم بود تا موسى به نزد ما بازگردد موسى گفت: اى هارون! وقتى ديدى آنها گمراه شدند چه چيز مانع تو شد كه از من پيروى نكنى؟ آيا امر مرا نافرمانى كردى
0.5402070879936218
*****************************
[هنگامی که موسی بازگشت، به هارون] گفت: ای هارون! وقتی دیدی آنان گمراه شدند چه چیز تو را مانع شد، از اینکه مرا [در برخورد شدید با گمراهان] پیروی کنی؟ آیا از فرمان من سرپیچی کردی؟ گفت: ای پسر مادرم! نه ریش مرا بگیر و نه سرم را، من ترسیدم که بگویی: میان بنی اسرائیل تفرقه و جدایی انداختی و سفارش مرا [در حفظ وحدت بنی اسرائیل] رعایت نکردی.
0.5390711426734924
*****************************
[موسى‌] گفت: «اى هارون، وقتى ديدى آنها گمراه شدند چه چيز مانع تو شد، كه از من پيروى كنى؟ آيا از فرمانم سر باز زدى؟» گفت: «اى پسر ما

In [ ]:

updated_message = [
        {"role": "system", "content": "I want you to act as a question answering bot which uses the context mentioned and answer in a concise manner in persian and doesn't make stuff up."
                                      "You will answer question based on the context - {}"
                                     },
        {"role": "user", "content": "Now I want you to answer this question {}."},
    ]

updated_message[0]['content'] = updated_message[0]['content'].format(retrieved_examples['text'])
updated_message[1]['content'] = updated_message[1]['content'].format(question)

completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=updated_message,
                                                      timeout=240, max_tokens=400, n=1, stop=None, temperature=0)

In [ ]:
completion['choices'][0]['message']['content']

'To apply for a new PAN card as an NRI, follow these steps:\n1. Visit the ABC app.\n2. Navigate to Services > NRI Pan Card > Apply New PAN.\n3. Select the required form of PAN card and proceed with the payment.\n4. Our team will contact you to request the necessary documents, including your passport (any country) or OCI card, passport size photograph, and overseas address proof with zip code (such as Indian NRO/NRE account statement, overseas bank statement, or utility bill).'